In [1]:
from finvizfinance.quote import finvizfinance
ticker = 'NFLX'
stock = finvizfinance(ticker)

In [2]:
news_df = stock.ticker_news()
news_df['Tickers'] = ticker
news_df

,Date,Title,Link,Tickers
0,2023-03-05 20:56:00,Netflix making live-action 'One Piece' from po...,https://finance.yahoo.com/news/netflix-making-...,NFLX
1,2023-03-05 15:06:00,Britain's Prince Harry invited to King Charles...,https://finance.yahoo.com/news/britains-prince...,NFLX
2,2023-03-05 08:38:00,Chris Rock Slaps Back in Netflixs First Live C...,https://finance.yahoo.com/m/40e4ae24-8fb3-3d6c...,NFLX
3,2023-03-05 00:32:00,Chris Rock unleashes on Will Smith and wife Ja...,https://finance.yahoo.com/news/chris-rock-unle...,NFLX
4,2023-03-03 15:44:00,Netflix is still Netflix as they produce live ...,https://finance.yahoo.com/video/netflix-still-...,NFLX
...,...,...,...,...
95,2023-02-12 10:31:00,2 Stocks That Are Fantastic Deals Right Now,https://finance.yahoo.com/m/6aa18f20-0dfb-359d...,NFLX
96,2023-02-12 07:31:00,"1 Growth Stock to Buy in 2023, and 1 to Avoid ...",https://finance.yahoo.com/m/28459ca2-4694-3433...,NFLX
97,2023-02-12 07:15:00,A Bull Market Is Coming: 1 FAANG Stock to Avoi...,https://finance.yahoo.com/m/f14b0c34-cc79-32c3...,NFLX
98,2023-02-11 17:05:00,Is Netflix Stock A Buy After Video Streamer's ...,https://finance.yahoo.com/m/815ee59a-c296-3fd4...,NFLX


In [3]:
outer_ratings_df = stock.ticker_outer_ratings()
outer_ratings_df

,Date,Status,Outer,Rating,Price
0,2023-01-23,Reiterated,Argus,Buy,$340 → $390
1,2023-01-20,Reiterated,Wolfe Research,Outperform,$366 → $417
2,2023-01-20,Reiterated,Wedbush,Outperform,$400 → $410
3,2023-01-20,Reiterated,The Benchmark Company,Sell,$225 → $250
4,2023-01-20,Reiterated,Robert W. Baird,Neutral,$275 → $325
5,2023-01-20,Reiterated,Pivotal Research Group,Buy,$375 → $400
6,2023-01-20,Reiterated,Piper Sandler,Neutral,$270 → $325
7,2023-01-20,Reiterated,Oppenheimer,Outperform,$400 → $415
8,2023-01-20,Reiterated,MoffettNathanson,Market Perform,$240 → $250
9,2023-01-20,Reiterated,Jefferies,Buy,$385 → $400


In [4]:
import yfinance as yf
ticker= get_data("amzn", start_date="12/04/2009", end_date="12/04/2019", index_as_date = True, interval="1wk")
ticker.info

NameError: name 'get_data' is not defined

In [ ]:
import yahoo_fin.stock_info as si
dow_list = si.tickers_nasdaq()
print("Tickers in NasdaQ:", len(dow_list))
dow_list

In [ ]:
import yahoo_fin.stock_info as si
si.get_analysts_info('nflx')

In [ ]:
from yahoo_fin.stock_info import *
import yfinance as yf
nasdaq = tickers_nasdaq()
nasdaq_list = nasdaq[:10]
for i in nasdaq_list:
    ticker = yf.download(i,interval='1m',period='1d',progress=False)
    ticker['Ticker'] = i
    print(ticker)

In [ ]:
from yahoo_fin.stock_info import *
import yfinance as yf
ticker = "ACE.BK"
data = yf.Ticker(ticker)
data.news

In [ ]:
from bs4 import BeautifulSoup
import requests


url = 'https://www.nasdaq.com/articles/will-roku-stock-break-hearts-this-week'
headers = {
    'User-Agent': 'Chrome/58.0.3029.110'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'lxml')
data = soup.find('h1', class_='jupiter22-c-hero-article-title')

if data:
    print(data.text)
else:
    print('Could not find the data')

In [ ]:
import unittest
from unittest.mock import patch,MagicMock
import sqlite3
import pandas as pd
import datetime as dt
import yfinance as yf
from pandas.testing import assert_frame_equal

def save_data_news(data):
    # connect to the database
    conn = sqlite3.connect('stock.sqlite')
    # save the data to the database
    data.to_sql('stock_news',con=conn,if_exists='append',index=False)
    # close the connection
    conn.close()
df2 = pd.DataFrame()
df1 = pd.DataFrame({'Datetime': ['2023-02-19'], 'Title':['ทอท.'], 'Link':['Test'], 'Body':['Test'], 'Ticker':['AOT']})
df2 = pd.concat([df2,df1],ignore_index=True)
save_data_news(df2)


Spatial

In [ ]:
import sqlite3
import pandas as pd
import datetime as dt
import yfinance as yf

content_news = []
conn = sqlite3.connect("stock.sqlite")
cur = conn.cursor()
query = "SELECT Ticker FROM stock_info"
r_df = pd.read_sql(query, conn)
list_db = r_df['Ticker'].values.tolist()
df2 = pd.DataFrame()

for i in list_db:
    query2 = "SELECT Datetime,ticker,body FROM stock_news WHERE `Ticker` = '%s'" % i
    news = pd.read_sql(query2, conn)
    if not(news.empty):
        df2 = pd.concat([df2,news],ignore_index=True)
    
df2

In [ ]:
import nltk
import spacy
 
# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [2]:
import locationtagger
from geopy.geocoders import Nominatim

# initializing sample text
sample_text = "India has very rich and vivid culture\
       widely spread from Kerala to Nagaland to Haryana to Maharashtra. " \
       "Delhi being capital with Mumbai financial capital.\
       Can be said better than some western cities such as " \
       " Munich, London etc. Pakistan and Bangladesh share its borders"

def getcity_to_latlong(text):
    # extracting entities.
    place_entity = locationtagger.find_locations(text = text)

    # calling the Nominatim tool
    loc = Nominatim(user_agent="GetLoc")
    address = pd.DataFrame()

    for i in place_entity.cities:
        getLoc = loc.geocode(i)
        ones = pd.DataFrame({'city':[i],'lat':[getLoc.latitude],'long':[getLoc.longitude]})
        # getting all cities
        address = pd.concat([address,ones],ignore_index=True)
    return address

getcity_to_latlong(sample_text)

,city,lat,long
0,Mumbai,19.078545,72.878176
1,Delhi,28.651718,77.221939
2,London,51.507336,-0.127650
3,Munich,48.137108,11.575382
4,Haryana,29.000000,76.000000


In [3]:
a = len(df2)
print(a)

40876


In [ ]:
place = pd.DataFrame()
count = 10000
a = len(df2)
#5529, 7218, 7249 error 
try:
    for i in range(10000,a):
        # if count < 5000:
        #     data = getcity_to_latlong(df2.iloc[i]['Body'])
        #     data['Datetime'] = df2.iloc[i]['Datetime']
        #     data['Ticker'] = df2.iloc[i]['Ticker']
        #     place = pd.concat([place,data],ignore_index=True)
        # if count >= 5000 and count < 10000:
        #     data = getcity_to_latlong(df2.iloc[i]['Body'])
        #     data['Datetime'] = df2.iloc[i]['Datetime']
        #     data['Ticker'] = df2.iloc[i]['Ticker']
        #     place = pd.concat([place,data],ignore_index=True)
        if count >= 10000 and count < 15000:
            data = getcity_to_latlong(df2.iloc[i]['Body'])
            data['Datetime'] = df2.iloc[i]['Datetime']
            data['Ticker'] = df2.iloc[i]['Ticker']
            place = pd.concat([place,data],ignore_index=True)
        # if count >= 15000 and count < 20000:
        #     data = getcity_to_latlong(df2.iloc[i]['Body'])
        #     data['Datetime'] = df2.iloc[i]['Datetime']
        #     data['Ticker'] = df2.iloc[i]['Ticker']
        #     place = pd.concat([place,data],ignore_index=True)
        # if count >= 20000:
        #     data = getcity_to_latlong(df2.iloc[i]['Body'])
        #     data['Datetime'] = df2.iloc[i]['Datetime']
        #     data['Ticker'] = df2.iloc[i]['Ticker']
        #     place = pd.concat([place,data],ignore_index=True)
        else:
            break
        count += 1
except:
    pass
place

In [23]:
# เพื่อ save ระวังข้อมูลซ้ำด้วยเด้อ
import sqlite3

address = place

conn = sqlite3.connect("stock.sqlite")
address.to_sql('stock_city',con=conn,if_exists='append',index=False)

1725

In [6]:
address = place
add = address.groupby(address.columns.tolist(),as_index=False).size()
add.rename(columns={'size': 'population'}, inplace=True)
add

,city,lat,long,population
0,Access,38.209249,-83.071005,1
1,Allen,40.807780,-84.057222,1
2,Almaty,43.236392,76.945728,1
3,America,39.783730,-100.445882,1
4,Bank,51.513105,-0.089375,2
5,Barron,45.423557,-91.845410,1
6,Bristol,51.453802,-2.597298,1
7,Calgary,51.046095,-114.065465,1
8,California,36.701463,-118.755997,1
9,Canada,61.066692,-107.991707,1


In [ ]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

df = add

geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
gdf = GeoDataFrame(df, geometry=geometry)   
gdf['Size'] = gdf['population'] * 20
#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(figsize=(20, 10),color='black', edgecolor='black')
a = gdf.plot(ax=base, marker='o', column='population', cmap = 'Reds', markersize='Size',legend=True)
a.set_facecolor("lightslategray")

In [ ]:
gdf

แปลภาษาไทย

In [4]:
import sqlite3
import pandas as pd
import datetime as dt
import yfinance as yf

content_news = []
conn = sqlite3.connect("stock.sqlite")
cur = conn.cursor()
query = "SELECT Ticker FROM stock_info WHERE `Index` = 'SET100'"
r_df = pd.read_sql(query, conn)
list_db = r_df['Ticker'].values.tolist()
thai = []
df2 = pd.DataFrame()
for j in list_db:
    query2 = "SELECT Datetime,ticker,body FROM stock_news WHERE `Ticker` = '%s'" % j
    news = pd.read_sql(query2, conn)
    if not(news.empty):
        df2 = pd.concat([df2,news],ignore_index=True)
    
df2

,Datetime,Ticker,Body
0,2023-02-06 00:00:00,AAV.BK,ผู้สื่อข่าวรายงานว่า วันนี้(6 ก.พ.) เป็นวันแรก...
1,2023-02-02 00:00:00,AAV.BK,นายกรภัทร วรเชษฐ์ ผู้อำนวยการ ฝ่ายวิจัยและการบ...
2,2023-02-01 00:00:00,AAV.BK,ADVERTISEMENT\nบริษัทหลักทรัพย์ ไอร่า จำกัด ระ...
3,2023-01-31 00:00:00,AAV.BK,ADVERTISEMENT\n บล.โนมูระ พัฒนสิน ระบุในบทวิเค...
4,2023-01-30 00:00:00,AAV.BK,อาการ “กลืนไม่เข้า…คายไม่ออก” ของเหล่าบรรดา “ผ...
...,...,...,...
7983,2023-02-23 00:00:00,WHA.BK,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน...
7984,2023-02-23 00:00:00,WHA.BK,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน...
7985,2023-02-22 00:00:00,WHA.BK,บล.ดาโอ ระบุในบทวิเคราะห์วันนี้ (22 ก.พ.66) ว...
7986,2023-02-22 00:00:00,WHA.BK,ผู้สื่อข่าวรายงานว่า บริษัท บีดับบลิวเอฟ (ไทย...


In [16]:
import translators as ts
import translators.server as tss
import pandas as pd

count = 0
a = len(df2)
data_thai = pd.DataFrame()
for i in range(a):
    try:
        data = pd.DataFrame({'Datetime':[df2.iloc[i]['Datetime']],'Ticker':[df2.iloc[i]['Ticker']],'Eng':[tss.google(df2.iloc[i]['Body'])]})
        # data['Datetime'] = df2.iloc[i]['Datetime']
        # data['Ticker'] = df2.iloc[i]['Ticker']
        data_thai = pd.concat([data_thai,data],ignore_index=True)
    except:
        pass
data_thai

,Datetime,Ticker,Eng
0,2023-02-06 00:00:00,AAV,Reporters reported that today (6 February) is ...
1,2023-02-02 00:00:00,AAV,"Mr. Kornpat Worachet, Director of Research and..."
2,2023-02-01 00:00:00,AAV,Advertisement \n Ira Securities Company Limite...
3,2023-01-31 00:00:00,AAV,Advertisement \n Nomura Pattanasin Securities...
4,2023-01-30 00:00:00,AAV,"Symptoms ""Swallow ... ""The guardian of Santira..."
...,...,...,...
91,2023-01-30 00:00:00,ADVANC,"The Thai stock market index is closed at 1,681..."
92,2023-01-30 00:00:00,ADVANC,Advertisement \n Heng Heng shares \n If we wan...
93,2023-01-25 00:00:00,ADVANC,"Mr. Warun Thep Watcharaporn, Head of Business ..."
94,2023-01-25 00:00:00,ADVANC,"Overall, the global economy seems to have redu..."


In [18]:
place = pd.DataFrame()
count = 0
a = len(data_thai)
for i in range(a):
    data = getcity_to_latlong(data_thai.iloc[i]['Eng'])
    data['Datetime'] = data_thai.iloc[i]['Datetime']
    data['Ticker'] = data_thai.iloc[i]['Ticker']
    place = pd.concat([place,data],ignore_index=True)
place

,city,lat,long,Datetime,Ticker
0,Phuket,7.936602,98.352929,2023-02-06 00:00:00,AAV
1,Egypt,26.254049,29.267547,2023-02-06 00:00:00,AAV
2,Cuba,23.013134,-80.832875,2023-02-06 00:00:00,AAV
3,Singapore,1.357107,103.819499,2023-02-06 00:00:00,AAV
4,Russia,64.686314,97.745306,2023-02-06 00:00:00,AAV
...,...,...,...,...,...
155,Phuket,7.936602,98.352929,2022-11-14 00:00:00,AAV
156,Chiang Mai,18.788278,98.985880,2022-11-14 00:00:00,AAV
157,Don Mueang,13.922654,100.600997,2022-11-14 00:00:00,AAV
158,Us,39.783730,-100.445882,2022-11-14 00:00:00,AAV


In [23]:
# เพื่อ save ระวังข้อมูลซ้ำด้วยเด้อ
import sqlite3

address = place

conn = sqlite3.connect("stock.sqlite")
address.to_sql('stock_city',con=conn,if_exists='append',index=False)

160

In [19]:
address = place
add = address.groupby(address.columns.tolist(),as_index=False).size()
add.rename(columns={'size': 'population'}, inplace=True)
add

,city,lat,long,Datetime,Ticker,population
0,Anan,43.354900,0.817728,2023-01-19 00:00:00,AAV,1
1,Argentina,-34.996496,-64.967282,2023-01-23 00:00:00,AAV,1
2,Argentina,-34.996496,-64.967282,2023-02-06 00:00:00,AAV,1
3,Atlanta,33.748992,-84.390264,2022-11-22 00:00:00,AAV,1
4,Au,-24.776109,134.755000,2023-01-19 00:00:00,AAV,1
...,...,...,...,...,...,...
146,Us,39.783730,-100.445882,2023-01-16 00:00:00,AAV,1
147,Us,39.783730,-100.445882,2023-01-17 00:00:00,AAV,1
148,Valentine,43.094907,0.704317,2023-01-08 00:00:00,AAV,1
149,Valentine,43.094907,0.704317,2023-01-24 00:00:00,AAV,1


In [ ]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

df = add

geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
gdf = GeoDataFrame(df, geometry=geometry)   
gdf['Size'] = gdf['population'] * 40
#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(figsize=(20, 10),color='black', edgecolor='black')
a = gdf.plot(ax=base, marker='o', column='population', cmap = 'Reds', markersize='Size',legend=True)
a.set_facecolor("lightslategray")

In [ ]:
# เพื่อ save ระวังข้อมูลซ้ำด้วยเด้อ
import sqlite3

address = place

conn = sqlite3.connect("stock.sqlite")
address.to_sql('stock_city',con=conn,if_exists='append',index=True)

In [ ]:
import pandas as pd
import sqlite3
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame


def plot_spatial(ticker):
    conn = sqlite3.connect("stock.sqlite")
    query = "SELECT city,lat,long FROM stock_city WHERE `Ticker` = '%s'" % ticker
    address = pd.read_sql(query, conn)
    add = address.groupby(address.columns.tolist(),as_index=False).size()
    add.rename(columns={'size': 'population'}, inplace=True)
    df = add

    geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
    gdf = GeoDataFrame(df, geometry=geometry)   
    gdf['Size'] = gdf['population'] * 40
    #this is a simple map that goes with geopandas
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    base = world.plot(figsize=(20, 10),color='black', edgecolor='black')
    a = gdf.plot(ax=base, marker='o', column='population', cmap = 'Reds', markersize='Size',legend=True)
    a.set_facecolor("lightslategray")


plot_spatial('AAV.BK')

In [ ]:
import plotly.express as px
import pandas as pd
import sqlite3
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

def plot_spatial(ticker):
    conn = sqlite3.connect("stock.sqlite")
    query = "SELECT city,lat,long FROM stock_city WHERE `Ticker` = '%s'" % ticker
    address = pd.read_sql(query, conn)
    add = address.groupby(address.columns.tolist(),as_index=False).size()
    add.rename(columns={'size': 'population'}, inplace=True)
    df = add

    fig = px.scatter_geo(df,lat="lat", lon="long", color="population",
                     hover_name="city", size="population",
                     projection="natural earth")
    return fig

plot_spatial('AAV.BK')

In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime

conn = sqlite3.connect("stock.sqlite")
query = "SELECT datetime FROM stock_table_hr WHERE datetime like '%+%' "
data = pd.read_sql(query, conn)

data = data.values.tolist()
fix = []
for i in data:
    fix.append(i[0].split('+')[0])
# Convert strings to datetime objects
dates = [datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S') for date_str in fix]
count=0
for j in dates:
    query1 = "UPDATE stock_table_hr SET datetime= ? WHERE datetime=?"
    cursor = conn.cursor()
    print(query1)
    cursor.execute(query1, (j,data[count][0]))
    conn.commit()
    count += 1


In [54]:
import translators as ts
import translators.server as tss
import locationtagger
import sqlite3
import pandas as pd
from datetime import datetime
from geopy.geocoders import Nominatim


def trans_set100(df2):
    count = 0
    a = len(df2)
    data_thai = pd.DataFrame()
    for i in range(a):
        try:
            data = pd.DataFrame({'Datetime':[df2.iloc[i]['Datetime']],'Ticker':[df2.iloc[i]['Ticker']],'Body':[tss.google(df2.iloc[i]['Body'])]})
            # data['Datetime'] = df2.iloc[i]['Datetime']
            # data['Ticker'] = df2.iloc[i]['Ticker']
            data_thai = pd.concat([data_thai,data],ignore_index=True)
        except:
            pass
    return data_thai

def getcity_and_latlong(text):
    # extracting entities.
    place_entity = locationtagger.find_locations(text = text)

    # calling the Nominatim tool
    loc = Nominatim(user_agent="GetLoc")
    address = pd.DataFrame({'city': [],'lat':[],'long':[]})

    for i in place_entity.cities:
        getLoc = loc.geocode(i)
        ones_city = pd.DataFrame({'city':[i],'lat':[getLoc.latitude],'long':[getLoc.longitude]})
        # getting all cities
        address = pd.concat([address,ones_city],ignore_index=True)
    return address
    
def get_latlong_for_all_content(df):
    place = pd.DataFrame()
    a = len(df)
    for i in range(a):
        data = getcity_and_latlong(df.iloc[i]['Body'])
        data['Datetime'] = df.iloc[i]['Datetime']
        data['Ticker'] = df.iloc[i]['Ticker']
        place = pd.concat([place,data],ignore_index=True)
    return place

def update_place(ticker):
    conn = sqlite3.connect("stock.sqlite")
    query_index = "SELECT `Index` FROM stock_info WHERE `Ticker` = '%s'" % ticker
    check_index = pd.read_sql(query_index, conn).values.tolist()[0][0]
    query_Dplace = "SELECT Datetime FROM stock_city WHERE `Ticker` = '%s'" % ticker
    check_Dplace = pd.read_sql(query_Dplace, conn).sort_values(by=['Datetime'],ascending=False).values.tolist()[0][0]
    query_news = "SELECT Datetime,ticker,body FROM stock_news WHERE datetime > '%s' and `Ticker` == '%s'" % (check_Dplace,ticker)
    get_news = pd.read_sql(query_news, conn)
    if check_index == 'SET100':
        print('This is set100')
        eng = trans_set100(get_news)
        place = get_latlong_for_all_content(eng)
    else:
        place = get_latlong_for_all_content(get_news)
    place.to_sql('stock_city',con=conn,if_exists='append',index=False)
    return place

update_place('ABNB')

,city,lat,long,Datetime,Ticker
0,Lucerne,47.050545,8.305468,2023-03-12 00:00:00,ABNB
1,Swiss,46.798562,8.231974,2023-03-12 00:00:00,ABNB
2,White House,38.897700,-77.036553,2023-03-11 00:00:00,ABNB
3,Argentina,-34.996496,-64.967282,2023-03-11 00:00:00,ABNB
4,London,51.507336,-0.127650,2023-03-11 00:00:00,ABNB
5,Paris,48.858890,2.320041,2023-03-11 00:00:00,ABNB
6,Samsung,59.927299,30.361444,2023-03-11 00:00:00,ABNB
7,Oracle,32.609970,-110.765184,2023-03-10 00:00:00,ABNB
8,Cobalt,37.546081,-90.287263,2023-03-10 00:00:00,ABNB
9,March,48.057857,7.779004,2023-03-09 00:00:00,ABNB
